In [1]:
import random
import time
import threading

In [2]:
def sleep(inter_num, interval):
    for i in range(inter_num):
        print("I am alife {}".format(i))
        time.sleep(interval)
t = threading.Thread(target=sleep, args=(5, 1))
t.start()

I am alife 0
I am alife 1
I am alife 2
I am alife 3
I am alife 4


In [12]:
# TODO: давайте починим output

'''
def sleep(inter_num , interval):
    for i in range(inter_num):
        print("I am alife {}".format(i))
        time.sleep(interval)

inter_num = 5
interval = 1

t = threading.Thread(target=sleep, args=(inter_num, interval))
t.start()

for i in range(inter_num):
    print("I am boss {}".format(i))
    time.sleep(interval)
'''
#рассинхрон ломает вывод
#исправление:
def sleep(inter_num , interval, lock):
    for i in range(inter_num):
        with lock:
            print("I am alife {}".format(i))
        time.sleep(interval)

inter_num = 5
interval = 0

lock = threading.Lock()

t = threading.Thread(target=sleep, args=(inter_num, interval, lock))
t.start()

for i in range(inter_num):
    with lock:
        print("I am boss {}".format(i))
    time.sleep(interval)


I am alife 0
I am alife 1
I am alife 2
I am alife 3
I am boss 0
I am boss 1
I am boss 2
I am alife 4
I am boss 3
I am boss 4


In [5]:
class Sleeper(threading.Thread):
    def __init__(self, name, inter_num, interval):
        super().__init__()
        self.name = name
        self.iter_num = inter_num
        self.interval = interval
    
    def run(self):
        for i in range(self.iter_num):
            print("I am alife {} my name {}".format(i, self.name))
            time.sleep(self.interval)

inter_num = 5
interval = 1
            
t = Sleeper("Ilya", inter_num, interval)
t.start()

for i in range(inter_num):
    print("I am boss {}".format(i))
    time.sleep(interval)

I am alife 0 my name Ilya
I am boss 0
I am alife 1 my name IlyaI am boss 1

I am alife 2 my name IlyaI am boss 2

I am alife 3 my name Ilya
I am boss 3
I am boss 4I am alife 4 my name Ilya



In [ ]:
# TODO: Создадим 6 потоков по числу преподов Vlad, Zarina, Misha, Ilya, Vlad, Anton

In [6]:
# join
class Sleeper(threading.Thread):
    def __init__(self, name, inter_num, interval):
        super().__init__()
        self.name = name
        self.iter_num = inter_num
        self.interval = interval
    
    def run(self):
        for i in range(self.iter_num):
            print("I am alife {} my name {}".format(i, self.name))
            time.sleep(self.interval)

inter_num = 5
interval = 1
            
t = Sleeper("Ilya", inter_num, interval)
t.start()
t.join()

for i in range(inter_num):
    print("I am boss {}".format(i))
    time.sleep(interval)

I am alife 0 my name Ilya
I am alife 1 my name Ilya
I am alife 2 my name Ilya
I am alife 3 my name Ilya
I am alife 4 my name Ilya
I am boss 0
I am boss 1
I am boss 2
I am boss 3
I am boss 4


In [ ]:
# TODO: Заставим преподов из задачки выше говорить последовательно

In [7]:
class DollarsAccount:
    def __init__(self, dollars_amount):
        self._dollars = dollars_amount

In [8]:
class Inflation(threading.Thread):
    def __init__(self, name, account, amount, iter_num):
        super().__init__()
        self.name = name
        self.account = account
        self.amount = amount
        self.iter_num = iter_num
    
    def run(self):
        print("My name {} account id {}".format(self.name, id(self.amount)))
        for i in range(self.iter_num):
            self.account._dollars -= self.amount
            
class DepositCap(threading.Thread):
    def __init__(self, name, account, amount, iter_num):
        super().__init__()
        self.name = name
        self.account = account
        self.amount = amount
        self.iter_num = iter_num
    
    def run(self):
        print("My name {} account id {}".format(self.name, id(self.amount)))
        for i in range(self.iter_num):
            self.account._dollars += self.amount

In [9]:
dac = DollarsAccount(50)
inflation = Inflation("inflation", dac, 1, 100000)      
deposit = DepositCap("DepositCap", dac, 1, 100000)

inflation.start()
deposit.start()
inflation.join()
deposit.join()
print(dac._dollars)

My name inflation account id 140703450305344
My name DepositCap account id 140703450305344
39530


In [15]:
class Inflation(threading.Thread):
    def __init__(self, name, account, amount, iter_num):
        super().__init__()
        self.name = name
        self.account = account
        self.amount = amount
        self.iter_num = iter_num
        self.lock = threading.Lock()
    
    def run(self):
        print("My name {} account id {}".format(self.name, id(self.amount)))
        for i in range(self.iter_num):
            self.lock.acquire()
            self.account._dollars -= self.amount
            self.lock.release()
            
class DepositCap(threading.Thread):
    def __init__(self, name, account, amount, iter_num):
        super().__init__()
        self.name = name
        self.account = account
        self.amount = amount
        self.iter_num = iter_num
        self.lock = threading.Lock()
    
    def run(self):
        print("My name {} account id {}".format(self.name, id(self.amount)))
        for i in range(self.iter_num):
            self.lock.acquire()
            self.account._dollars += self.amount
            self.lock.release()

In [18]:
dac = DollarsAccount(50)
inflation = Inflation("inflation", dac, 5, 100000)      
deposit = DepositCap("DepositCap", dac, 5, 100000)

inflation.start()
deposit.start()
inflation.join()
deposit.join()
print(dac._dollars)

#Почему не сработало?

My name inflation account id 140703450305472
My name DepositCap account id 140703450305472
50


In [21]:
class Inflation(threading.Thread):
    def __init__(self, name, account, amount, iter_num, lock):
        super().__init__()
        self.name = name
        self.account = account
        self.amount = amount
        self.iter_num = iter_num
        self.lock = lock
    
    def run(self):
        print("My name {} account id {}".format(self.name, id(self.amount)))
        for i in range(self.iter_num):
            with self.lock:
                self.account._dollars -= self.amount
            
class DepositCap(threading.Thread):
    def __init__(self, name, account, amount, iter_num, lock):
        super().__init__()
        self.name = name
        self.account = account
        self.amount = amount
        self.iter_num = iter_num
        self.lock = lock
    
    def run(self):
        print("My name {} account id {}".format(self.name, id(self.amount)))
        for i in range(self.iter_num):
            with self.lock:
                self.account._dollars += self.amount
            
dac = DollarsAccount(50)
lock = threading.Lock()
inflation = Inflation("inflation", dac, 5, 100000, lock)      
deposit = DepositCap("DepositCap", dac, 5, 100000, lock)

inflation.start()
deposit.start()
inflation.join()
deposit.join()
print(dac._dollars)

#для синхронизации разных потоков используем один и тот же lock

My name inflation account id 140703450305472
My name DepositCap account id 140703450305472
50


In [ ]:
import threading, time

a = 5
alock = threading.Lock()
b = 5
block = threading.Lock()

def thread1calc():
    print("Thread1 acquiring lock a")
    alock.acquire()
    time.sleep(5)
#захват лока б, не отпуская лок а
    print("Thread1 acquiring lock b")
    block.acquire()
    time.sleep(5)
    a += 5
    b += 5

    print("Thread1 releasing both locks")
    block.release()
    alock.release()

def thread2calc():
    print ("Thread2 acquiring lock b")
    block.acquire()
    time.sleep(5)

    print("Thread2 acquiring lock a")
    alock.acquire()
    time.sleep(5)
    a += 10
    b += 10

    print("Thread2 releasing both locks")
    block.release()
    alock.release()

t1 = threading.Thread(target = thread1calc)
t1.start()

t2 = threading.Thread(target = thread2calc)
t2.start()

t1.join()
t2.join()

#deadlock
#solution - в t2 сначала захватить лок а, потом лок б. Или добавить третий лок с

Thread1 acquiring lock aThread2 acquiring lock b

Thread1 acquiring lock bThread2 acquiring lock a



In [ ]:
import threading, time

a = 5
alock = threading.Lock()
b = 5
block = threading.Lock()
clock = threading.Lock()

def thread1calc():
    global a, b
    with clock:    
        print("Thread1 acquiring lock a")
        alock.acquire()
        time.sleep(5)
        print("Thread1 acquiring lock b")
        block.acquire()
        time.sleep(5)
        a += 5
        b += 5

        print("Thread1 releasing both locks")
        block.release()
        alock.release()

def thread2calc():
    global a, b
    with clock:
        print ("Thread2 acquiring lock b")
        block.acquire()
        time.sleep(5)

        print("Thread2 acquiring lock a")
        alock.acquire()
        time.sleep(5)
        a += 10
        b += 10

        print("Thread2 releasing both locks")
        block.release()
        alock.release()

t1 = threading.Thread(target = thread1calc)
t1.start()

t2 = threading.Thread(target = thread2calc)
t2.start()

t1.join()
t2.join()